In [1]:
!pip install -q -U langchain transformers bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 852.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/

In [3]:
!pip install langchain-community langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00


In [4]:
import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [14]:
import os

os.environ['HF_TOKEN'] = "api_key"
os.environ['HUGGINGFACEHUB_API_TOKEN'] = "api_key"

In [12]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write)

In [181]:
# using quantization to reduce space usage and optimize performance
# if quantization is not done the RAM will shoot up and loading of Mistral 7B can't be done
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model_4bit = AutoModelForCausalLM.from_pretrained( "mistralai/Mistral-7B-Instruct-v0.1", device_map="auto",quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [182]:
# configuring the llm pipeline

pipeline_inst = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=2500,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

In [183]:
llm = HuggingFacePipeline(pipeline=pipeline_inst) # getting the llm ready

In [184]:
template = """<s>[INST] You are an respectful and helpful assistant, respond always be precise, assertive and politely answer in few words conversational english.
Answer the question below from context below :
{question}****
"""

In [185]:
# function to chat witn the llm
def generate_response(question):
  prompt = PromptTemplate(template=template, input_variables=["question","context"])
  llm_chain = LLMChain(prompt=prompt, llm=llm)
  response = llm_chain.run({"question":question})
  return response

In [186]:
import pandas as pd

#load the dataset
data = pd.read_csv("/content/movie dataset.csv")

In [187]:
import re

def create_new_description(title, description):
    query = (
        f"Title: {title}\n"
        f"Description: {overview}\n"
        f"Using the title and description provided, generate a new creative description."
    )

    creative_description = generate_response(query)
    creative_description = creative_description.split('****')[1] #separate question from answer

    return creative_description.strip()

def detect_genre(title, description, actual_genre):
    query = (
        f"Imagine you are an expert in identifying the genre of movies.\n"
        f"Title: {title}\n"
        f"Description: {overview}\n"
        f"Classify this movie into one of the following genres (science fiction, adventure, action, crime, drama, thriller, animation, family)"
    )

    predicted_genre = generate_response(query)
    predicted_genre = predicted_genre.split('****')[1] #separate question from answer

    val = 0
    actual = actual_genre.split(',')
    for i in actual:
      if i.strip().lower() in predicted_genre.strip().lower():
        val = 1
        break

    return (predicted_genre.strip(), val)

def plot_twist(title, description):
    query = (
        f"Imagine you are an expert in generating twists in movies.\n"
        f"Title: {title}\n"
        f"Description: {overview}\n"
        f"Generate one plot twists and one alternate story ending for this movie"
    )

    twist = generate_response(query)
    twist = twist.split('****')[1] #separate question from answer

    return twist.strip()

In [188]:
movies = []
total = correct = 0
for i in range(20):
    title = data['title'][i]
    overview = data['overview'][i]
    cast = data['cast'][i]
    genres = data['genres'][i]

    new_genre, val = detect_genre(title, overview, genres)
    creative_description = create_new_description(title, overview)
    plot_twists = plot_twist(title, overview)

    if val == 1:
      correct += 1
    total += 1

    #Store the movie details along with the new generated genre, description and twist
    movie = {
        "title": title,
        "overview": overview,
        "cast": cast,
        "genres": genres,
        "new_genre": new_genre,
        "creative_description": creative_description,
        "plot_twist": plot_twists
    }

    # Append movie dictionary to movies list
    movies.append(movie)

movies

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'title': 'Avatar',
  'overview': 'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.',
  'cast': 'Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez',
  'genres': 'Action,Adventure,Fantasy,Science Fiction',
  'new_genre': 'Answer: Avatar falls under the genre of science fiction, adventure and thriller.',
  'creative_description': 'New creative description:\n"Embark on a thrilling adventure as a brave paraplegic Marine navigates the treacherous landscapes of the moon Pandora, torn between following military orders and saving an endangered alien civilization. Can you make the right choices to protect both humanity and the universe?"',
  'plot_twist': 'Plot twist: The Marine discovers that the alien species he was protecting were planning on invading Earth.\n\nAlternate Story Ending: The Marine decides to stay on Pandora and helps the ali

In [180]:
print('Accuracy:', correct / total)

Accuracy: 0.8


In [73]:
print(movies[0]['title'])
print(movies[0]['new_genre'])
print(movies[0]['creative_description'])
print(movies[0]['plot_twist'])
print(movies[0]['overview'])
print(movies[0]['cast'])
print(movies[0]['genres'])

print(correct/total)

Avatar
Answer: Science fiction/adventure/animation
Avatar: A Paraplegic Marine's Mission to Pandora
As the last of the humans on Pandora, the paraplegic Marine, named Jake Sully, is tasked with protecting the planet's indigenous alien species, called the Na'vi. However, when Jake discovers that the humans who came before him have been mining Pandora's resources and polluting the planet, he must decide whether to follow the orders and destroy the Na'vi or risk being exiled from the planet and losing the one thing he ever felt whole. With the help of his new Na'vi guide, Neytiri, Jake embarks on a journey to find a way to stop the humans and save Pandora. Along the way, he learns more about himself and the Na'vi's way of life. Will Jake's mission to destroy the Na'vi change his mind, and will he become a hero to both species? Avatar is a visually stunning film that explores themes of love, betrayal, and the importance of protecting the environment.
1. Avatar ends on a cliffhanger with Ja